In [1]:
import os

# Generated by Perplexity

def read_text_files(directory_path):
    """
    Reads all .txt files in the specified directory and returns their contents as a list of strings.

    :param directory_path: The path to the directory containing .txt files
    :return: A list of strings where each string is the content of a .txt file
    """
    txt_contents = []
    # Check if the directory exists
    if not os.path.isdir(directory_path):
        print(f"The directory {directory_path} does not exist.")
        return txt_contents

    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        # Check for .txt extension
        if filename.endswith(".txt"):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Open and read the contents of the file
            try:
                with open(file_path, 'r') as file:
                    txt_contents.append(file.read())
            except IOError as e:
                print(f"Failed to read file {filename}: {e}")

    return txt_contents


In [ ]:
from langchain_anthropic import ChatAnthropic

# Set your Anthropic API key
# Make sure to keep your API key secure, consider using environment variables
os.environ["ANTHROPIC_API_KEY"] = "A"  # replace with your actual key

# Initialize the Anthropic chat model (Claude)
llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0,
    max_tokens=4096,
    timeout=None,
    max_retries=2,
)

In [ ]:
#local_model_path = "/Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf"

In [3]:
embedding_data_dir = "/Users/maxhuang/projects/llm/langchain-book-examples/data"

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import DirectoryLoader
from llama_cpp import Llama
# Load the Llama model for embeddings and chat
#llama_model = Llama(model_path=local_model_path, verbose=False, n_ctx=131000)

In [5]:
embedding_model_dir = "/Users/maxhuang/projects/llm/models/all-MiniLM-L6-v2"

In [9]:
#from read_text_files import read_text_files

from langchain.embeddings import HuggingFaceEmbeddings

# Load HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_dir)

# Replace OpenAI embeddings with Llama embeddings
vectorstore = FAISS.from_texts(read_text_files(embedding_data_dir), embedding=embedding_model)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Define a wrapper for the Llama model to use as a chat interface
#class ChatLlamaWrapper:
#    def __call__(self, prompt_text):
#        return llama_model(prompt_text)["choices"][0]["text"]
# Define a wrapper for the Llama model to use as a chat interface
class ChatLlamaWrapper:
    def __call__(self, prompt_text):
        # Convert the ChatPromptValue to a plain string
        if isinstance(prompt_text, dict):
            prompt_text = prompt_text.get("text", "")
        elif hasattr(prompt_text, "to_string"):
            prompt_text = prompt_text.to_string()
        elif not isinstance(prompt_text, str):
            raise ValueError("Input to Llama model must be a string.")
        
        # Get response from LLM
        response = llm(prompt_text)
        
        # Handle AIMessage response
        if hasattr(response, 'content'):
            return response.content
        # Handle dictionary response
        elif isinstance(response, dict) and "choices" in response:
            return response["choices"][0]["text"]
        # Return raw response as fallback
        return str(response)


model = ChatLlamaWrapper()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("who tried to define what Chemistry is?"))
print(chain.invoke("What kinds of equipment are in a chemistry laboratory?"))
print(chain.invoke("What is Austrian School of Economics?"))
print(chain.invoke("Why do people engage in sports?"))
print(chain.invoke("What is the effect of body chemistry on exercise?"))


Based on the given context, there are a few mentions related to defining the term "chemistry":

1. In 1730, Georg Ernst Stahl defined "chemistry" as "the art of resolving mixed, compound, or aggregate bodies into their principles; and of composing such bodies from those principles."

2. In 1837, Jean-Baptiste Dumas considered "chemistry" to refer to "the science concerned with the laws and effects of molecular forces."

3. In 1947, the definition evolved to mean "the science of substances: their structure, their properties, and the reactions that change them into other substances" - a characterization accepted by Linus Pauling.

4. In 1998, the definition was broadened by Professor Raymond Chang to mean "the study of matter and the changes it undergoes."

So the context mentions Georg Ernst Stahl, Jean-Baptiste Dumas, Linus Pauling, and Raymond Chang as people who tried to define what chemistry is over different time periods.
Based on the given context, the only relevant information ab